In [1]:
import random
import sys
from matplotlib import pyplot as plt
sys.path.append('../..')

from model import run
from model import plot_utils as pu
from model import processing
from model.amm.omnipool_amm import OmnipoolState
from model.amm.agents import Agent
from model.amm.trade_strategies import omnipool_arbitrage
from model.amm.global_state import GlobalState, fluctuate_prices, historical_prices

# same seed, same parameters = same simulation result
random.seed(42)

assets = {
    'HDX': {'usd price': 0.0267, 'weight': 0.10},
    'DAI': {'usd price': 1, 'weight': 0.5},
    'DOT': {'usd price': 5, 'weight': 0.4},
}

lrna_price_usd = 22.22222222
initial_omnipool_tvl = 100000
liquidity = {}
lrna = {}
min_price = 0.00001
time_steps = 20

for tkn, info in assets.items():
    liquidity[tkn] = initial_omnipool_tvl * info['weight'] / info['usd price']
    lrna[tkn] = initial_omnipool_tvl * info['weight'] / lrna_price_usd

initial_state = GlobalState(
    pools={
        'Omnipool': OmnipoolState(
            tokens={
                tkn: {'liquidity': liquidity[tkn], 'LRNA': lrna[tkn]} for tkn in assets
            },
            lrna_fee=0,
            asset_fee=0,
            preferred_stablecoin = 'DAI',
        )
    },
    agents={
        'Arbitrageur': Agent(
            holdings={tkn: 1000000000000 for tkn in list(assets.keys()) + ['LRNA']}, 
            trade_strategy=omnipool_arbitrage('Omnipool')
        )
    },
    # evolve_function=historical_prices({
    #     i+1: {'HDX': assets['HDX']['usd price'] * (min_price + 1 - (1 - min_price) / time_steps * i)} for i in range(time_steps)
    # }),
    external_market={tkn: assets[tkn]['usd price'] for tkn in assets}
)

# lrna_price = []
# hdx_price = []
# dot_price = []
# for i in range(1, 100):
#     final_hdx_price = 0.0267 * (1 - i * (1 / 100))
#     hdx_price.append(final_hdx_price)
#     next_state = initial_state.copy()
#     next_state._evolve_function=historical_prices([{'HDX': 0.0267}, {'HDX': 0.0267}, {'HDX': final_hdx_price}])
#     events = run.run(next_state, time_steps=2, silent=True)
#     final_state = events[-1]['state'].pools['Omnipool']
#     lrna_price.append(final_state.liquidity['USD'] / final_state.lrna['USD'])
#     dot_price.append(final_state.usd_price('DOT'))

# plt.figure(figsize=(20, 5))
# plt.plot(hdx_price, lrna_price)
# plt.gca().invert_xaxis()
# plt.title('LRNA vs HDX spot')

# plt.figure(figsize=(20, 5))
# plt.plot(hdx_price, dot_price)
# plt.gca().invert_xaxis()
# plt.title('DOT vs HDX spot')

# crash HDX
crash_state = initial_state.copy()
crash_state.evolve_function = None
# crash_state.external_market['HDX'] *= 0.05
crash_state._evolve_function=historical_prices([{'HDX': 0.0267}, {'HDX': 0.0267}, {'HDX': 0.001335}])
worse_crash_state = crash_state.copy()
# worse_crash_state.external_market['HDX'] *= 0.2
worse_crash_state._evolve_function=historical_prices([{'HDX': 0.0267}, {'HDX': 0.0267}, {'HDX': 0.000267}])

# time_steps = 20
# events = run.run(initial_state, time_steps=time_steps)

# plt.figure(figsize=(20, 5))
# plt.plot(pu.get_datastream(events, asset='HDX'),
#    [event['state'].pools['Omnipool'].liquidity['DAI'] / event['state'].pools['Omnipool'].lrna['DAI'] for event in events]
# )
# plt.gca().invert_xaxis()
crash_events = run.run(crash_state, time_steps=2)
# pu.plot(crash_events, pool='Omnipool', prop='usd_price', key='HDX')
end_state_1 = run.run(crash_state, time_steps=2, silent=True)[-1]['state']
end_state_2 = run.run(worse_crash_state, time_steps=2, silent=True)[-1]['state']

# print ('datastream===', pu.get_datastream(events, asset='HDX'))
# print(initial_state)
# print('====================================')
# print('HDX crashes 95%')
# print('====================================')
# print(end_state_1)
# print('====================================')
# print('HDX crashes 99%')
# print('====================================')
# print(end_state_2)
start_omnipool = initial_state.pools['Omnipool']
crash_omnipool = end_state_1.pools['Omnipool']
worse_crash_omnipool = end_state_2.pools['Omnipool']
print(f"starting price of LRNA: {start_omnipool.liquidity['DAI'] / start_omnipool.lrna['DAI']}")
print(f"LRNA price: HDX crashes by 95%: {crash_omnipool.liquidity['DAI'] / crash_omnipool.lrna['DAI']}")
print(f"LRNA price: HDX crashes by 99%: {worse_crash_omnipool.liquidity['DAI'] / worse_crash_omnipool.lrna['DAI']}")

Starting simulation...
Execution time: 0.0 seconds.
starting price of LRNA: 22.222222220000001044453286539005179269877775247419
LRNA price: HDX crashes by 95%: 18.905538300220476872117148344961986641918101700512
LRNA price: HDX crashes by 99%: 18.402222220382001980693340490404213195899078519211


In [2]:
# print(processing.value_assets(initial_state.external_market, crash_state.agents['Arbitrageur']))

# print(processing.value_assets(initial_state.external_market, crash_state.agents['Arbitrageur']))

# print(processing.value_assets(initial_state.external_market, crash_state.agents['Arbitrageur']))